# Make a Choropleth

### ipyleaflet example
From https://ipyleaflet.readthedocs.io/en/latest/api_reference/choropleth.html

In [1]:
import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json',
    'us-states.json',
     json.load)

unemployment = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/US_Unemployment_Oct2012.csv',
    'US_Unemployment_Oct2012.csv',
     pd.read_csv)

unemployment =  dict(zip(unemployment['State'].tolist(), unemployment['Unemployment'].tolist()))

layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=unemployment,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

m = ipyleaflet.Map(center = (43,-100), zoom = 4)
m.add_layer(layer)
m

Map(center=[43, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Implement ipyleaflet choropleth
Using data originating from a shapefile

Imports

In [2]:
%matplotlib inline

import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, Marker, Popup, GeoJSON, basemaps
from branca.colormap import linear
import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import numpy as np

Import shapefile data

In [3]:
path = '../Supplementary_data/Machine_learning_with_ODC/example_training_data.shp'

# Load input shapefile
input_data = gpd.read_file(path)

# Plot the first five rows
input_data.head()

,classnum,geometry
0,112,"POLYGON ((-1521875.000 -3801925.000, -1521900...."
1,111,"POLYGON ((-1557925.000 -3801125.000, -1557950...."
2,111,"POLYGON ((-1555325.000 -3800000.000, -1555200...."
3,111,"POLYGON ((-1552925.000 -3800950.000, -1552925...."
4,111,"POLYGON ((-1545475.000 -3800000.000, -1544325...."


In [4]:
# Convert to WGS 84 and GeoJSON format
gdf_wgs84 = input_data.to_crs(epsg=4326) # geopandas.geodataframe.GeoDataFrame
data = gdf_wgs84.__geo_interface__       # Feature Collection
feature_layer = GeoJSON(data=data)

# Get centroid to focus map on
lon1, lat1, lon2, lat2  = gdf_wgs84.total_bounds
lon = (lon1 + lon2) / 2
lat = (lat1 + lat2) / 2

Import table data and convert to correct format 

The keys must be strings corresponding to id in the geojson
If the values to be coloured on are no evenly spread, re-map

In [5]:
# Remap bad classes for visualisation
classes_uni = list(input_data.classnum.unique())
classes_clean = list(range(0,len(classes_uni)))

# zip together to make a dictionary
classes_dict =  dict(zip(classes_uni, classes_clean))

# create new input_data column
input_data['classnum_clean']=input_data['classnum'].map(classes_dict)
input_data

,classnum,geometry,classnum_clean
0,112,"POLYGON ((-1521875.000 -3801925.000, -1521900....",0
1,111,"POLYGON ((-1557925.000 -3801125.000, -1557950....",1
2,111,"POLYGON ((-1555325.000 -3800000.000, -1555200....",1
3,111,"POLYGON ((-1552925.000 -3800950.000, -1552925....",1
4,111,"POLYGON ((-1545475.000 -3800000.000, -1544325....",1
...,...,...,...
212,215,"POLYGON ((-1539025.000 -3846475.000, -1539025....",2
213,111,"POLYGON ((-1540400.000 -3847200.000, -1540400....",1
214,111,"POLYGON ((-1541625.000 -3847575.000, -1541700....",1
215,111,"POLYGON ((-1535025.000 -3845425.000, -1535075....",1


In [6]:
# Option 2 - get data from the polygon collection

# Get values to colour by as a list
classes = input_data['classnum_clean'].tolist()

# Get id values as a list
input_data['id']=input_data.index
keys = input_data['id'].tolist()

# zip together to make a dictionary
the_dict =  dict(zip(keys, classes))

# Convert ids to strings
the_dict2 = {str(key): val for key, val in the_dict.items()}

In [7]:
layer = ipyleaflet.Choropleth(
    geo_data=data,
    choro_data=the_dict2,
    colormap=linear.YlOrRd_04, 
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '1'}
)

In [8]:
# Print the ipyleaflet map
basic_map = Map(zoom=10,
               center=(lat, lon))
display(basic_map)

# Add GeoJSON layer to map
basic_map.add_layer(layer)

Map(center=[-34.110424911963065, 115.32099948027002], controls=(ZoomControl(options=['position', 'zoom_in_text…

### Not used

In [10]:
# station_nb is a dataframe with values grouped by c_ar

geop = r'arrondissements.geojson'    
map.choropleth(
    geo_data =data,
    name='choropleth',
    data = station_nb,
    columns = ['c_ar','count'],
    key_on='features.properties.c_ar',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2
)

AttributeError: type object 'map' has no attribute 'choropleth'

In [ ]:
geo_json_data = json.load(open('../aglayers.geojson'))

In [ ]:
geo_json_data['properties']['classnum']

In [ ]:
data['features'][0]['properties']['class']

In [ ]:
# Option 1 - Make an attribute table with id and classnum and export it in seperatley
csv = pd.read_csv('../new_table2.csv')
data_dict =  dict(zip(csv['id'].tolist(), csv['classnum'].tolist()))
#print(data_dict.items())

### map_shapefile

In [ ]:
%matplotlib inline

import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, Marker, Popup, GeoJSON, basemaps
from branca.colormap import linear
import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import numpy as np

In [ ]:
path = '../Supplementary_data/Machine_learning_with_ODC/example_training_data.shp'

# Load input shapefile
input_data = gpd.read_file(path)


In [ ]:
# Convert to WGS 84 and GeoJSON format
gdf_wgs84 = input_data.to_crs(epsg=4326) # geopandas.geodataframe.GeoDataFrame
data = gdf_wgs84.__geo_interface__       # Feature Collection
feature_layer = GeoJSON(data=data)

# Get centroid to focus map on
lon1, lat1, lon2, lat2  = gdf_wgs84.total_bounds
lon = (lon1 + lon2) / 2
lat = (lat1 + lat2) / 2

Import table data and convert to correct format 

The keys must be strings corresponding to id in the geojson
If the values to be coloured on are no evenly spread, re-map

In [ ]:
# Remap bad classes for visualisation
classes_uni = list(input_data.classnum.unique())
classes_clean = list(range(0,len(classes_uni)))
classes_dict =  dict(zip(classes_uni, classes_clean)) # zip together to make a dictionary
input_data['classnum_clean']=input_data['classnum'].map(classes_dict) # create new input_data column

In [ ]:
# Create the dictionary to colour map by
classes = input_data['classnum_clean'].tolist() # Get values to colour by as a list
input_data['id']=input_data.index # Get id values as a list
keys = input_data['id'].tolist()
the_dict =  dict(zip(keys, classes)) # zip together to make a dictionary
the_dict2 = {str(key): val for key, val in the_dict.items()} # Convert ids to strings

In [ ]:
# Create the choropleth
layer = ipyleaflet.Choropleth(
    geo_data=data,
    choro_data=the_dict2,
    colormap=linear.YlOrRd_04, 
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '1'}
)

In [ ]:
# Print the ipyleaflet map
basic_map = Map(zoom=10,
               center=(lat, lon))
display(basic_map)

# Add GeoJSON layer to map
basic_map.add_layer(layer)

### map_shapefile edits

In [ ]:
# Import required packages
import folium  
import math
import numpy as np
import ipywidgets
import matplotlib as mpl
from pyproj import Proj, transform
from IPython.display import display
from ipyleaflet import Map, Marker, Popup, GeoJSON, basemaps
from skimage import exposure
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from datetime import datetime
import calendar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

# I have added
import ipyleaflet
from branca.colormap import linear

In [ ]:
def _degree_to_zoom_level(l1, l2, margin=0.0):
    
    """
    Helper function to set zoom level for `display_map`
    """
    
    degree = abs(l1 - l2) * (1 + margin)
    zoom_level_int = 0
    if degree != 0:
        zoom_level_float = math.log(360 / degree) / math.log(2)
        zoom_level_int = int(zoom_level_float)
    else:
        zoom_level_int = 18
    return zoom_level_int


# map_shapefile with choropleth
def map_shapefile(gdf, 
                  colour_class = 'user_defined', # the column to use to colour on
                  remap = 'remap', # defaults to remapping 'colour_class' for better visualisation, if unwanted specify 'colour_class'
                  weight=2, 
                  colormap=mpl.cm.YlOrRd, 
                  basemap=basemaps.Esri.WorldImagery, 
                  default_zoom=None,
                  hover_col=None,
                  hover_prefix=''):
    
    
#     def n_colors(n, colormap=colormap):
#         data = np.linspace(0.0,1.0,n)
#         c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
#         return c

#     def data_to_colors(data, colormap=colormap):
#         c = [mpl.colors.rgb2hex(d[0:3]) for 
#              d in colormap(mpl.colors.Normalize()(data))]
#         return c 
    
    def on_hover(event, id, properties):
        with dbg:
            text = properties.get(hover_col, '???')
            lbl.value = f'{hover_col}: {text}'
            # print(properties)
  
    # Convert to WGS 84 and GeoJSON format
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    data = gdf_wgs84.__geo_interface__    
    
#     # For each feature in dataset, append colour values
#     n_features = len(data['features'])
#     colors = n_colors(n_features)
    
#     for feature, color in zip(data['features'], colors):
#         feature['properties']['style'] = {'color': color, 
#                                           'weight': weight, 
#                                           'fillColor': color, 
#                                           'fillOpacity': 1.0}

    # Remap bad classes for visualisation
    classes_uni = list(gdf[colour_class].unique())
    classes_clean = list(range(0,len(classes_uni)))
    classes_dict =  dict(zip(classes_uni, classes_clean)) # zip together to make a dictionary
    gdf['remap']=gdf[colour_class].map(classes_dict) # create new gdf column
    
    # Create the dictionary to colour map by
    # How to make this 'classnum_clean' or 'colour_class'
    classes = gdf[remap].tolist() # Get values to colour by as a list
    gdf['id']=gdf.index # Get id values as a list
    keys = gdf['id'].tolist()
    the_dict =  dict(zip(keys, classes)) # zip together to make a dictionary
    the_dict2 = {str(key): val for key, val in the_dict.items()} # Convert ids to strings

    # Get centroid to focus map on
    lon1, lat1, lon2, lat2  = gdf_wgs84.total_bounds
    lon = (lon1 + lon2) / 2
    lat = (lat1 + lat2) / 2
    
    if default_zoom is None:
        
        # Calculate default zoom from latitude of features
        default_zoom = _degree_to_zoom_level(lat1, lat2, margin=-0.5)
    
    # Plot map 
    m = Map(center=(lat, lon), 
            zoom=default_zoom, 
            basemap=basemap, 
            layout=dict(width='800px', height='600px'))
    
    # Create the choropleth
    choropleth = ipyleaflet.Choropleth(
    geo_data=data,
    choro_data=the_dict2,
    colormap=linear.YlOrRd_04, 
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '1'})
    
#     # Add GeoJSON layer to map
#     feature_layer = GeoJSON(data=data)
#     m.add_layer(feature_layer)
    
    # Add Choropleth layer to map
    m.add_layer(choropleth)
    
    # If a column is specified by `hover_col`, print data from the
    # hovered feature above the map
    if hover_col:        
        lbl = ipywidgets.Label()
        dbg = ipywidgets.Output()        
        choropleth.on_hover(on_hover)
        display(lbl)
      
    # Display the map
    display(m)

In [ ]:
path = '../Supplementary_data/Machine_learning_with_ODC/example_training_data.shp'

# Load input shapefile
input_data = gpd.read_file(path)

map_shapefile(input_data,colour_class='classnum',hover_col='classnum')

### map_shapefile as is

In [ ]:
# Import required packages
import folium  
import math
import numpy as np
import ipywidgets
import matplotlib as mpl
from pyproj import Proj, transform
from IPython.display import display
from ipyleaflet import Map, Marker, Popup, GeoJSON, basemaps
from skimage import exposure
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from datetime import datetime
import calendar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
def map_shapefile(gdf, 
                  weight=2, 
                  colormap=mpl.cm.YlOrRd, 
                  basemap=basemaps.Esri.WorldImagery, 
                  default_zoom=None,
                  hover_col=None,
                  hover_prefix=''):
    
    """
    Plots a geopandas GeoDataFrame over an interactive ipyleaflet 
    basemap. Optionally, can be set up to print selected data from 
    features in the GeoDataFrame. 
    
    Last modified: October 2019
    
    Parameters
    ----------  
    gdf : geopandas.GeoDataFrame
        A GeoDataFrame containing the spatial features to be plotted 
        over the basemap
    weight : float or int, optional
        An optional numeric value giving the weight that line features
        will be plotted as. Defaults to 2; larger numbers = thicker
    colormap : matplotlib.cm, optional
        An optional matplotlib.cm colormap used to style the features
        in the GeoDataFrame. Features will be coloured by the order
        they appear in the GeoDataFrame. Defaults to the `YlOrRd` 
        colormap.
    basemap : ipyleaflet.basemaps object, optional
        An optional ipyleaflet.basemaps object used as the basemap for 
        the interactive plot. Defaults to `basemaps.Esri.WorldImagery`
    default_zoom : int, optional
        An optional integer giving a default zoom level for the 
        interactive ipyleaflet plot. Defaults to 13
    hover_col : str, optional
        An optional string giving the name of any column in the
        GeoDataFrame you wish to have data from printed above the 
        interactive map when a user hovers over the features in the map.
        Defaults to None which will not print any data. 

    """
    
    def n_colors(n, colormap=colormap):
        data = np.linspace(0.0,1.0,n)
        c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
        return c

    def data_to_colors(data, colormap=colormap):
        c = [mpl.colors.rgb2hex(d[0:3]) for 
             d in colormap(mpl.colors.Normalize()(data))]
        return c 
    
    def on_hover(event, id, properties):
        with dbg:
            text = properties.get(hover_col, '???')
            lbl.value = f'{hover_col}: {text}'
            # print(properties)
  
    # Convert to WGS 84 and GeoJSON format
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    data = gdf_wgs84.__geo_interface__    
    
    # For each feature in dataset, append colour values
    n_features = len(data['features'])
    colors = n_colors(n_features)
    
    for feature, color in zip(data['features'], colors):
        feature['properties']['style'] = {'color': color, 
                                          'weight': weight, 
                                          'fillColor': color, 
                                          'fillOpacity': 1.0}

    # Get centroid to focus map on
    lon1, lat1, lon2, lat2  = gdf_wgs84.total_bounds
    lon = (lon1 + lon2) / 2
    lat = (lat1 + lat2) / 2
    
    if default_zoom is None:
        
        # Calculate default zoom from latitude of features
        default_zoom = _degree_to_zoom_level(lat1, lat2, margin=-0.5)
    
    # Plot map 
    m = Map(center=(lat, lon), 
            zoom=default_zoom, 
            basemap=basemap, 
            layout=dict(width='800px', height='600px'))
    
    # Add GeoJSON layer to map
    feature_layer = GeoJSON(data=data)
    m.add_layer(feature_layer)
    
    # If a column is specified by `hover_col`, print data from the
    # hovered feature above the map
    if hover_col:        
        lbl = ipywidgets.Label()
        dbg = ipywidgets.Output()        
        feature_layer.on_hover(on_hover)
        display(lbl)
      
    # Display the map
    display(m)


In [ ]:
path = '../Supplementary_data/Machine_learning_with_ODC/example_training_data.shp'
field = 'classnum'
product = 'ls8_nbart_geomedian_annual'
year = 2015
feature_stats = 'mean'

In [ ]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

In [ ]:
# Create custom colourmap a single colour
cmap = matplotlib.colors.ListedColormap(["#4ca334"])

# Plot training data in an interactive map
map_shapefile(input_data, colormap=cmap, hover_col=field)

### Folium

In [ ]:
import pandas as pd


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo, # geojson
    name='choropleth',
    data=state_data, # a table with in built id, 'State' and 'Unemployment'
    columns=['State', 'Unemployment'], # named columns in state_data
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
potato_json = json.load(open('../potato.geojson'))
potato_csv = pd.read_csv('../potato.csv')


In [ ]:
import pandas as pd


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[-34.05, 115.17], zoom_start=11)

folium.Choropleth(
    geo_data=potato_json, # geojson
    name='choropleth',
    data=potato_csv, # a table with in built id, 'State' and 'Unemployment'
    columns=['potato_no', 'classnum'], # named columns in state_data
    key_on='feature.properties.potato_no',
    fill_color='PuRd_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='classnum'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
potato_json

In [ ]:
potato_json[feature][properties][potato_no]